In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import datetime
import csv
import os
from sklearn.metrics import r2_score, mean_squared_error
from IPython.display import display

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from modules.prediction import load_all_data
from modules.prediction import precrime_train_test_split
from modules.prediction import load_splits
from modules.prediction import create_all_splits
from modules.prediction import sample_model
from modules.fancy_time_series import fancy_time_series_model
from modules.eval_model import eval_predictions

In [3]:
crime_data = load_all_data()
splits = load_splits()
train_test_data = create_all_splits(crime_data, splits)

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
X_train_fine, X_test_fine, y_train_fine, y_test_fine = train_test_data['fine']
X_train_coarse, X_test_coarse, y_train_coarse, y_test_coarse = train_test_data['coarse']
X_train_2016, X_test_2016, y_train_2016, y_test_2016 = train_test_data['2016']

In [5]:
y_ts_fine = fancy_time_series_model(X_train_fine, y_train_fine, X_test_fine, y_test_fine)
y_ts_coarse = fancy_time_series_model(X_train_coarse, y_train_coarse, X_test_coarse, y_test_coarse)
y_ts_2016 = fancy_time_series_model(X_train_2016, y_train_2016, X_test_2016, y_test_2016)

In [6]:
eval_predictions(X_test_fine, y_test_fine, y_ts_fine)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -2.1, RMSE =     0.055, RMSE (%) =  1977.185
Rape:             R2 =     -1.6, RMSE =     0.123, RMSE (%) =   884.016
Robbery:          R2 =      3.2, RMSE =     0.352, RMSE (%) =   304.105
FelonyAssault:    R2 =      4.0, RMSE =     0.367, RMSE (%) =   334.084
Burglary:         R2 =      2.6, RMSE =     0.348, RMSE (%) =   315.924
GrandLarceny:     R2 =     15.2, RMSE =     0.516, RMSE (%) =   205.752
GrandLarcenyAuto: R2 =      2.4, RMSE =     0.249, RMSE (%) =   422.572
Fraud:            R2 =      0.8, RMSE =     0.185, RMSE (%) =   578.384
Forgery:          R2 =      2.0, RMSE =     0.184, RMSE (%) =   618.552
Arson:            R2 =     -2.1, RMSE =     0.095, RMSE (%) =  1197.076
Drugs:            R2 =      4.2, RMSE =     0.200, RMSE (%) =   556.938
Weapons:          R2 =      2.4, RMSE =     0.177, RMSE

In [7]:
eval_predictions(X_test_coarse, y_test_coarse, y_ts_coarse)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -1.7, RMSE =     0.050, RMSE (%) =  2114.411
Rape:             R2 =     -1.2, RMSE =     0.123, RMSE (%) =   868.470
Robbery:          R2 =      3.9, RMSE =     0.347, RMSE (%) =   304.614
FelonyAssault:    R2 =      5.2, RMSE =     0.377, RMSE (%) =   321.838
Burglary:         R2 =      2.0, RMSE =     0.333, RMSE (%) =   325.360
GrandLarceny:     R2 =     14.7, RMSE =     0.524, RMSE (%) =   201.744
GrandLarcenyAuto: R2 =      2.5, RMSE =     0.235, RMSE (%) =   441.849
Fraud:            R2 =      0.3, RMSE =     0.177, RMSE (%) =   601.274
Forgery:          R2 =      1.7, RMSE =     0.177, RMSE (%) =   630.242
Arson:            R2 =     -1.5, RMSE =     0.091, RMSE (%) =  1240.591
Drugs:            R2 =      4.0, RMSE =     0.187, RMSE (%) =   579.150
Weapons:          R2 =      3.7, RMSE =     0.183, RMSE

In [8]:
eval_predictions(X_test_2016, y_test_2016, y_ts_2016)

------------------------------------------------------------------
Four-hour buckets:
------------------------------------------------------------------
Homicide:         R2 =     -1.8, RMSE =     0.045, RMSE (%) =  2403.798
Rape:             R2 =     -1.4, RMSE =     0.114, RMSE (%) =   912.278
Robbery:          R2 =      3.2, RMSE =     0.311, RMSE (%) =   339.816
FelonyAssault:    R2 =      4.8, RMSE =     0.381, RMSE (%) =   311.632
Burglary:         R2 =      0.0, RMSE =     0.286, RMSE (%) =   376.725
GrandLarceny:     R2 =     12.2, RMSE =     0.515, RMSE (%) =   206.721
GrandLarcenyAuto: R2 =      0.4, RMSE =     0.196, RMSE (%) =   529.180
Fraud:            R2 =      0.1, RMSE =     0.156, RMSE (%) =   669.338
Forgery:          R2 =      3.1, RMSE =     0.198, RMSE (%) =   556.387
Arson:            R2 =     -1.7, RMSE =     0.071, RMSE (%) =  1618.622
Drugs:            R2 =      1.7, RMSE =     0.175, RMSE (%) =   624.060
Weapons:          R2 =      4.1, RMSE =     0.184, RMSE